# SHAP Visualization & Interpretation

## Overview

This notebook presents a comprehensive visualization suite for **SHAP (SHapley Additive exPlanations)** analysis results from our cognitive load prediction model. SHAP values provide model-agnostic explanations by quantifying each feature's contribution to individual predictions.

### What You'll Find Here:
- **SHAP Summary Plots**: Global feature importance rankings and directional impact analysis
- **SHAP Cluster Analysis**: Identification of distinct explanation patterns across participants
- **Feature Importance Metrics**: Random Forest feature importance complementing SHAP insights
- **Cluster Assignments**: Participant-level segmentation based on explanation patterns

### How to Use This Notebook:
1. **Run cells sequentially** to display all available visualizations
2. Cells are **robust to missing files** and will indicate if artifacts are not found
3. Each visualization includes **captions and interpretations** suitable for reports and presentations
4. All figures are publication-ready with high DPI settings

---

## Background: Understanding SHAP Values

**SHAP (SHapley Additive exPlanations)** is a game-theoretic approach to explain machine learning model predictions. It provides:

- **Local Explanations**: How each feature contributed to a specific prediction
- **Global Insights**: Overall feature importance across the entire dataset
- **Directional Impact**: Whether high/low feature values increase or decrease predicted cognitive load

### Our Model Context:
- **Model Type**: Random Forest Classifier
- **Prediction Task**: Binary classification of cognitive load (High vs. Low)
- **Features**: Behavioral metrics, interaction patterns, and task-specific variables
- **Goal**: Identify which behavioral signals most reliably predict cognitive load states

### Interpretation Guidelines:
- **Positive SHAP values** → Push prediction toward "High Load"
- **Negative SHAP values** → Push prediction toward "Low Load"  
- **Magnitude** → Strength of the feature's impact
- **Color coding** (in beeswarm plots) → Original feature value (red = high, blue = low)

---

In [1]:
# Helper imports and lookup paths
from pathlib import Path
from IPython.display import display, Image, HTML, Markdown
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.dpi'] = 150

SEARCH_DIRS = [
    Path('../figures/shap'),
    Path('../results/interpretation'),
    Path('../results/modeling'),
]

def find_file(names):
    """Find first existing file from list of candidate names."""
    for d in SEARCH_DIRS:
        for n in names:
            p = d / n
            if p.exists():
                return p
    return None

def show_image(p, caption=None, width=900):
    """Display image with optional caption."""
    if p and p.exists():
        display(HTML(
            f"<div><img src='{p}' width='{width}'/>"
            f"<p style='font-size:90%'>{caption or p.name}</p></div>"
        ))
    else:
        print(f'Not found: {p}')

## 1. SHAP Summary Plots

These two complementary visualizations provide different perspectives on feature importance:

### 📊 SHAP Summary Bar Plot
Shows **mean absolute SHAP values** for each feature, ranking them by overall predictive power. This tells us *which features matter most* for the model's decisions, regardless of direction.

### 🎯 SHAP Beeswarm Plot
Shows the **distribution of SHAP values** across all samples. Key insights:
- **X-axis**: SHAP value (impact on model output)
- **Y-axis**: Features (ordered by importance)
- **Color**: Original feature value (red = high, blue = low)
- **Patterns**: Reveals how feature magnitude affects predictions

**What to Look For:**
- Features at the top are most influential
- Wide spreads indicate variable impact across samples
- Color gradients show if high values push toward High or Low load predictions

---

In [2]:
# Find and display SHAP summary plots
bar = find_file(['shap_summary_bar.png', 'shap_summary_barplot.png'])
bees = find_file(['shap_summary_beeswarm.png', 'shap_summary_beeswarmplot.png'])

print('SHAP bar path:', bar)
print('SHAP beeswarm path:', bees)

if bar:
    show_image(
        bar, 
        caption='Figure: SHAP summary bar (mean absolute SHAP values). '
                'Features are ranked by average impact on model output; '
                'higher values indicate more predictive power for High Load.'
    )
else:
    print('SHAP summary bar not found.')

if bees:
    show_image(
        bees, 
        caption='Figure: SHAP beeswarm. Each point represents one sample-feature SHAP value; '
                'color shows the original feature value. The plot reveals how feature '
                'magnitude directionally affects predictions.'
    )
else:
    print('SHAP beeswarm not found.')

SHAP bar path: ../figures/shap/shap_summary_bar.png
SHAP beeswarm path: ../figures/shap/shap_summary_beeswarm.png


## Local Explanation Example: SHAP Value Analysis

This section provides a combined interpretation of SHAP-based global feature importance using bar plots (mean absolute SHAP values) and beeswarm plots (directional feature impact).

### 📊 SHAP Bar Plot: Mean |SHAP| Values

**Top Drivers of Cognitive Load:**
- **Budget Management Stress** → Highest impact; repeated price checking and cost adjustments strongly predict high load
- **Constraint Violation Rate** → Invalid actions (breaking scheduling rules) indicate cognitive struggle
- **Idle Time Ratio** → High idle periods reflect planning confusion or decision paralysis

**Mid-Tier Contributors:** Scheduling difficulty, planning time ratio, multitasking load, and recovery efficiency reflect multi-step decision complexity.

**Lower-Impact Features:** Form-related metrics, exploration breadth, and user traits show minimal discriminative power.

### 🎨 SHAP Beeswarm Plot: Directional Impact

Each dot represents one trial. Color indicates feature value (blue = low, red = high). X-axis shows effect on predicted load.

**High Values → Higher Load:** Red dots on the positive side for budget stress, constraint violations, idle time, scheduling difficulty, and multitasking load all push predictions toward high cognitive load—these are "struggle signals."

**Low Values → Lower Load:** Blue dots on the negative side for recovery efficiency and form hesitation; high form efficiency reduces predicted load (matches low-complexity Tasks 1-2).

**Minimal Predictors:** Exploration breadth, rapid hovers, decision uncertainty, and traits cluster near zero with low predictive value.

### 🎯 Summary

The model correctly identifies **planning difficulty**, **rule violations**, and **budget management struggles** as primary cognitive load indicators, aligning with NASA-TLX ratings and Task 3 complexity. Form interactions show minimal impact, reflecting their task-specific nature. No spurious features dominate—the model learns **theoretically-grounded patterns** for cognitive load detection.

## 2. SHAP Cluster Analysis (PCA Visualization)

This analysis identifies **distinct subgroups** of participants based on their SHAP explanation patterns.

### Methodology:
1. Extract SHAP value vectors for each sample (participant-task combination)
2. Apply **k-means clustering** to identify groups with similar explanation patterns
3. Visualize clusters using **PCA dimensionality reduction**

### Why This Matters:
- **Heterogeneity**: Not all participants' cognitive load follows the same behavioral patterns
- **Personalization**: Different user groups may require different UI/UX interventions
- **Validity**: Clusters can validate task design (e.g., Task 3 should cluster separately if truly higher complexity)

### Interpretation:
- **Well-separated clusters** → Distinct explanation profiles exist
- **Cluster composition** → Check if clusters align with tasks, participant demographics, or load levels
- **Centroid analysis** → Identify which features most distinguish each cluster

---

In [3]:
# Find and display SHAP clusters PCA plot
pca = find_file(['shap_clusters_pca.png', 'shap_clusters_pca_plot.png'])

if pca:
    show_image(
        pca, 
        caption='Figure: PCA of SHAP vectors with k-means clusters. '
                'Points are samples; color shows cluster membership derived from SHAP patterns, '
                'indicating groups with similar model explanations.'
    )
else:
    print('SHAP clusters PCA not found.')

## SHAP Clustering (PCA 2D)

SHAP explanation vectors projected into 2D PCA space and grouped using KMeans clustering. Each point represents a single trial, positioned based on **similarity of SHAP contribution patterns** rather than raw behavioral values.

### 🔍 Cluster Interpretation

**Cluster 0 — Low/Moderate Cognitive Load Pattern (blue)**

Dense, compact grouping on the left side of PCA space with stable SHAP signatures. Dominated by trials where `scheduling_difficulty`, `constraint_violation_rate`, and `budget_management_stress` are low, while `recovery_efficiency` and `form_efficiency` show stronger negative SHAP influence. Corresponds mostly to **Tasks 1 & 2** (low/medium load). Reflects efficient operation with fewer interaction conflicts.

**Cluster 1 — High Cognitive Load Pattern (teal)**

Clearly separated on the right side with higher PCA1 values. Contains primarily **Task 3** trials and high TLX (>60) samples. SHAP vectors show strong positive contributions from `constraint_violation_rate`, `budget_management_stress`, and `idle_time_ratio`, plus increased influence from `multitasking_load` and `drag_attempts`. Captures high-load behavioral signature: more mistakes, switching, constraint conflicts, and elevated motor/visual workload.

### 🎯 Significance

Clusters emerge from **SHAP values** (model reasoning), not raw behavior. Clean separation indicates the model learned two distinct cognitive-behavioral regimes: one efficient and smooth, one conflict-heavy and resource-demanding. Provides strong **convergent validity** for metrics, TLX scores, and model predictions.

## 3. Feature Importances (Random Forest)

While SHAP provides **directional, local explanations**, Random Forest feature importances offer a complementary **global importance** perspective based on how often and how effectively features are used in decision trees.

### How RF Feature Importance Works:
- Measures **mean decrease in impurity** (Gini importance)
- Higher values → Feature creates more informative splits
- Based on **tree structure**, not individual predictions

### SHAP vs. RF Importance:
| Metric | SHAP | RF Importance |
|--------|------|---------------|
| **Scope** | Local + Global | Global only |
| **Direction** | Signed (positive/negative) | Unsigned (magnitude only) |
| **Basis** | Game theory (Shapley values) | Tree split quality |
| **Best For** | Understanding individual predictions | Overall feature ranking |

### Using Both Together:
- **Agreement** → High confidence in feature importance
- **Disagreement** → May indicate feature interactions or non-linear effects
- **Complementary insights** → Use SHAP for "how" and RF for "how often"

---

In [7]:
# Find and display feature importances
fi_csv = find_file([
    'feature_importances.csv',
    'feature_importances_ultrarealistic_summary.csv'
])

if fi_csv:
    df_fi = pd.read_csv(fi_csv)
    display(df_fi.head(30))
    display(Markdown(
        '**Caption:** Feature importances (Random Forest). '
        'Higher values indicate greater average contribution to splits.'
    ))
else:
    print('Feature importances CSV not found.')

,Unnamed: 0,mean_importance,std_importance
0,constraint_violation_rate,0.211153,0.009895
1,budget_management_stress,0.174965,0.008091
2,idle_time_ratio,0.117370,0.011846
3,scheduling_difficulty,0.097125,0.009405
4,multitasking_load,0.063168,0.008073
5,planning_time_ratio,0.053080,0.006873
6,recovery_efficiency,0.052724,0.007204
7,mouse_entropy_avg,0.033865,0.005595
8,form_hesitation_index,0.031634,0.004899
9,form_efficiency,0.028602,0.003221


**Caption:** Feature importances (Random Forest). Higher values indicate greater average contribution to splits.

## Feature Importance Analysis: Random Forest

Mean feature importance (averaged over LOUO folds) and standard deviation reflect how much each feature contributed to reducing prediction error in the Random Forest model.

### 🔥 Highest-Importance Features

**1. Constraint Violation Rate (0.211)** — Most important predictor; repeated invalid actions (overlapping schedules, impossible placements) indicate cognitive overload.

**2. Budget Management Stress (0.175)** — Frequent price-checking loops and budget recalculations signal decision strain and resource overload.

**3. Idle Time Ratio (0.117)** — High idle time reflects decision pauses, confusion, or planning delays.

**4. Scheduling Difficulty (0.097)** — Failed drag placements and planning conflicts directly reflect Task 3 complexity.

**5. Multitasking Load (0.063)** — Excessive panel switching captures UI fragmentation and cognitive switching costs.

These top 5 features explain the majority of predictive power through: planning strain, invalid operations, budget uncertainty, context switching, and hesitation.

### ⚙ Mid-Importance Features

**Planning Time Ratio (0.053)**, **Recovery Efficiency (0.053)**, **Mouse Entropy (0.034)**, and **Form Hesitation Index (0.032)** supplement high-level signals and help distinguish borderline cases (TLX ~55–62).

### 🧪 Low-Importance Features

Form efficiency, drag attempts, filter optimization, exploration breadth, action density, decision uncertainty, and user traits (skill, cautiousness) contributed minimally. **Zip Code Struggle (0.0029)** had the lowest importance, aligning with Task 1's low-load nature.

### 🎯 Summary

The Random Forest prioritizes **planning difficulty**, **constraint violations**, **budget stress**, **idle periods**, and **multitasking demands** as primary cognitive load indicators, strongly aligning with SHAP and TLX analyses. Lower-importance features (form metrics, personality traits) have minimal influence, demonstrating that the model focuses on **interaction struggle**, not incidental behavior.

## 4. SHAP Cluster Assignments & Metadata

This table provides **detailed cluster membership** for each participant-task combination, enabling:

### Analysis Opportunities:
- **Cluster Profiling**: What characteristics define each cluster?
- **Task-Cluster Relationships**: Do specific tasks concentrate in certain clusters?
- **Validation**: Does cluster membership align with self-reported cognitive load (NASA-TLX)?
- **Outlier Detection**: Samples that don't fit well into any cluster




In [9]:
# Find and display SHAP cluster assignments
cluster_csv = find_file([
    'shap_clusters_assignments.csv',
    'shap_cluster_labels_with_meta.csv',
    'participant_shap_clusters.csv'
])

if cluster_csv:
    dfc = pd.read_csv(cluster_csv)
    display(dfc.head(50))
    display(Markdown(
        '**Caption:** SHAP cluster labels. Each row is a sample (participant-task) '
        'with assigned cluster; examine cluster composition for intuition.'
    ))
else:
    print('No SHAP cluster labels CSV found.')

,participantId,task_id,tlx,High_Load,shap_cluster
0,p-0001,task_1_form,28.3,0,0
1,p-0001,task_2_product,40.0,0,0
2,p-0001,task_3_travel,71.7,1,1
3,p-0002,task_1_form,28.3,0,0
4,p-0002,task_2_product,33.3,0,0
5,p-0002,task_3_travel,63.3,1,1
6,p-0003,task_1_form,26.7,0,0
7,p-0003,task_2_product,78.3,1,0
8,p-0003,task_3_travel,50.0,0,0
9,p-0004,task_1_form,25.0,0,0


**Caption:** SHAP cluster labels. Each row is a sample (participant-task) with assigned cluster; examine cluster composition for intuition.

## SHAP Clustering (PCA 2D)

SHAP explanation vectors projected into 2D PCA space and grouped using KMeans clustering. Each point represents a single trial, positioned based on **similarity of SHAP contribution patterns** rather than raw behavioral values.

### 🔍 Cluster Interpretation

**Cluster 0 — Low/Moderate Cognitive Load Pattern (blue)**

Dense, compact grouping on the left side of PCA space with stable SHAP signatures. Dominated by trials where `scheduling_difficulty`, `constraint_violation_rate`, and `budget_management_stress` are low, while `recovery_efficiency` and `form_efficiency` show stronger negative SHAP influence. Corresponds mostly to **Tasks 1 & 2** (low/medium load). Reflects efficient operation with fewer interaction conflicts.

**Cluster 1 — High Cognitive Load Pattern (teal)**

Clearly separated on the right side with higher PCA1 values. Contains primarily **Task 3** trials and high TLX (>60) samples. SHAP vectors show strong positive contributions from `constraint_violation_rate`, `budget_management_stress`, and `idle_time_ratio`, plus increased influence from `multitasking_load` and `drag_attempts`. Captures high-load behavioral signature: more mistakes, switching, constraint conflicts, and elevated motor/visual workload.

### 🎯 Significance

Clusters emerge from **SHAP values** (model reasoning), not raw behavior. Clean separation indicates the model learned two distinct cognitive-behavioral regimes: one efficient and smooth, one conflict-heavy and resource-demanding. Provides strong **convergent validity** for metrics, TLX scores, and model predictions.